In [1]:
#https://eurodatacube.com/notebooks/contributions/FAIRiCUBE/FAIRiCUBE_data_access_demonstration.ipynb


%matplotlib inline


# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers
from xcube_sh.sentinelhub import SentinelHub
from xcube_sh.viewer import ViewerServer

# xcube imports
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.geom import clip_dataset_by_geometry

# Various utilities
import json
import matplotlib
import pandas as pd
import numpy as np
import xarray as xr
import shapely.geometry
import IPython.display
import zarr
from io import StringIO



import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
# Various utilities
import json
import xarray as xr
import shapely.geometry
import IPython.display
import zarr

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")






In [2]:
#hhttps://eurodatacube.com/notebooks/curated/EDC_xcube_generator_service.ipynb


x1 = 6.5  # degree
y1 = 49.7  # degree
x2 = 6.8  # degree
y2 = 49.8  # degree

bbox = x1, y1, x2, y2

resolution = 10
bbox = BBox(bbox=bbox, crs=CRS.WGS84)
bbox_size = bbox_to_dimensions(bbox, resolution=resolution)
print(f"Image shape at {resolution} m resolution: {bbox_size} pixels")

IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

Image shape at 10 m resolution: (2196, 1044) pixels


<IPython.display.GeoJSON object>

In [5]:
#https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B05', 'B08', 'B11', 'SCL', 'CLD'],
                         tile_size=[512, 512],
                         bbox=bbox,
                         spatial_res=10,
                         time_range=['2021-06-01', '2022-02-28'],
                         time_period='2D')

request_collector = Observers.request_collector()
cube = open_cube(cube_config, observer=request_collector)
cube

<xarray.Dataset>
Dimensions:    (time: 137, lat: 1, lon: 1, bnds: 2)
Coordinates:
  * lat        (lat) float64 54.7
  * lon        (lon) float64 11.5
  * time       (time) datetime64[ns] 2021-06-02 2021-06-04 ... 2022-03-01
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(137, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B04        (time, lat, lon) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    B05        (time, lat, lon) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    B08        (time, lat, lon) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    B11        (time, lat, lon) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    CLD        (time, lat, lon) uint8 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    SCL        (time, lat, lon) uint8 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
Attributes: (12/13)
    Conventions:               CF-1.7
    title:                     S2L2A Data Cube Subset
    history:                   [{'program': 'xcube_sh.chunkstore.SentinelHubC...
    date_created:              2023-03-02T14:42:20.658213
    time_coverage_start:       2021-06-01T00:00:00+00:00
    time_coverage_end:         2022-03-02T00:00:00+00:00
    ...                        ...
    time_coverage_resolution:  P2DT0H0M0S
    geospatial_lon_min:        6.5
    geospatial_lat_min:        49.7
    geospatial_lon_max:        16.5
    geospatial_lat_max:        59.7
    processing_level:          L2A

In [10]:
request_collector

In [4]:
cube.B04.isel(time=6).plot(figsize=[15,6], vmax=0.2);

Failed to fetch data from Sentinel Hub after 31.63940691947937 seconds and 200 retries
HTTP status code was 400


SentinelHubError: 400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process

In [14]:
sat_image = cube.B04

In [15]:
print (sat_image)
#https://docs.xarray.dev/
#https://github.com/sentinel-hub/eo-learn/blob/master/examples/land-cover-map/SI_LULC_pipeline.ipynb


<xarray.DataArray 'B04' (time: 137, lat: 1, lon: 1)>
dask.array<open_dataset-f20f2a230370157d6655779a5ce6571aB04, shape=(137, 1, 1), dtype=float32, chunksize=(1, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 54.7
  * lon      (lon) float64 11.5
  * time     (time) datetime64[ns] 2021-06-02 2021-06-04 ... 2022-03-01
Attributes:
    sample_type:   FLOAT32
    units:         reflectance
    wavelength:    664.75
    wavelength_a:  664.6
    wavelength_b:  664.9
    bandwidth:     31.0
    bandwidth_a:   31
    bandwidth_b:   31
    resolution:    10


In [ ]:
# NDVI: (B08 - B04)/(B08 + B04)

NDVI=(cube.B08 -cube.B04)/(cube.B08 +cube.B04)

i = 6  # time
NDVI.isel(time=i).plot(figsize=[15,6], vmax=1, cmap=plt.cm.Greens);
print ("done")



#plt.show()

In [1]:
# NDVI: (B08 - B04)/(B08 + B04)
  
NDVI=(cube.B08 -cube.B04)/(cube.B08 +cube.B04)

t = air.isel(time=slice(0, 365 * 4, 250))

g_NDVI=t.plot(figsize=[15,6], vmax=1,col ="time", col_warp=3, cmap=plt.cm.Greens);

print ("done")

NameError: name 'cube' is not defined

In [6]:
#https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/

#START--------------------------
#Collection name: UrbanAtlas2018_10m_raster_v1
#Collection id:  27c95f8f-fb6b-46f2-8f84-2ce7761eeccd


cube_config = CubeConfig(dataset_name='UrbanAtlas2018_10m_raster_v1',
                         band_names=['B1'],
                         tile_size=[512, 512],
                         bbox=bbox,
                         spatial_res=10,
                         time_range=['2021-06-01', '2022-02-28'],
                         time_period='2D')

request_collector = Observers.request_collector()
cube = open_cube(cube_config, observer=request_collector)
cube

<xarray.Dataset>
Dimensions:    (time: 137, lat: 1, lon: 1, bnds: 2)
Coordinates:
  * lat        (lat) float64 54.7
  * lon        (lon) float64 11.5
  * time       (time) datetime64[ns] 2021-06-02 2021-06-04 ... 2022-03-01
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(137, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B1         (time, lat, lon) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    title:                     UrbanAtlas2018_10m_raster_v1 Data Cube Subset
    history:                   [{'program': 'xcube_sh.chunkstore.SentinelHubC...
    date_created:              2023-03-02T14:45:42.104797
    time_coverage_start:       2021-06-01T00:00:00+00:00
    time_coverage_end:         2022-03-02T00:00:00+00:00
    time_coverage_duration:    P274DT0H0M0S
    time_coverage_resolution:  P2DT0H0M0S
    geospatial_lon_min:        6.5
    geospatial_lat_min:        49.7
    geospatial_lon_max:        16.5
    geospatial_lat_max:        59.7